In [2]:
import re
import json
import requests
#hash this as we'll be using it a lot
def get_page(url,params={}):
    return session_requests.get(
        url,
        params=params,
        headers = dict(referer=url))

session_requests = requests.session()
login_url = 'https://www.vinted.fr/member/general/login?ref_url='
tt = session_requests.get(login_url).content.decode('utf-8')
pp = re.compile('name="csrf-token" content="(.*?)"')
mm = pp.search(tt)

from vinted_creds import creds
payload = creds
payload["authenticity_token"] =  mm.group(1)

result = session_requests.post( 'https://www.vinted.fr/member/general/login.json',
                               json=json.dumps(payload),
                               headers = dict(referer=login_url)
                              )

In [3]:
result

<Response [422]>

In [5]:
url = 'https://www.vinted.fr/api/v2/users/20263980/items'

stuff = []
for k in range(1,7):
    print(k)
    r = get_page(url, 
                params={'page': k, 'per_page': 48})
    stuff.append( r.json())

1
2
3
4
5
6


you should really get the max_page from in the firts json batch
['pagination']

In [6]:
stuff = [x['items'] for x in stuff]
import itertools
df = list(itertools.chain.from_iterable(stuff))

In [8]:
its = [ (x['view_count'], x['title'] + x['description']) for x in df ]
its = [ (x['view_count'], x['title'] ) for x in df ]
sum( [ float(x['price_numeric']) for x in df ] )

961.5999999999999

In [9]:
from datetime import datetime
def str2date(ss):
    date = ss.split('+')[0]
    return  datetime.strptime(date, '%Y-%m-%dT%H:%M:%S')

def date2offset(ss):
    start = str2date('2018-12-02T15:53:11+01:00')
    
    return  (datetime.now() - str2date(ss) ).days

xx = [ (x['view_count'], 
        date2offset(x['created_at_ts']),
        x['title'] 
        ) for x in df ]

In [11]:
xx.sort(reverse=True)
xx[:20]

[(92, 1, 'pull ralph lauren femme bleu marine XS '),
 (58, 26, 'veste petit bateau marine fille '),
 (56, 44, 'manteau bleu marine petit bateau fille laine (78%)'),
 (48, 51, 'gants nike femme'),
 (45, 26, 'polo lacoste bleu marine femme '),
 (45, 20, 'basket nike femme (dual fusion X) 38'),
 (45, 20, 'Chaussures Nike Free 5.0 femme 38.5'),
 (42, 10, 'Super bonne paye '),
 (41, 30, 'tee shirt superdry bleu femme '),
 (41, 11, 'Kawaii le livre 100% Japon '),
 (40, 11, 'Mirogolo '),
 (32, 10, 'polo lacoste rose framboise femme 34'),
 (29, 400, 'Veste Jennifer'),
 (28, 22, '211 idées pour devenir un garçon génial '),
 (28, 11, 'Les énigmes de sherlock holmes'),
 (27, 29, 'Manteau ooxoo garçon marine en laine '),
 (26, 44, 'Gants répétons fille rose poudré'),
 (24, 11, 'Les goûters philo'),
 (24, 7, 'Boule et bill - Sos Planète '),
 (23, 22, 'Nos étoiles contraires - John Green')]

In [13]:
with open('classement.txt','w') as fp:
    fp.write( '\n'.join(['%d %d %s'%x for x in xx]) )